In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import os
import pickle
from collections import OrderedDict

import nept

from loading_data import get_data
from utils_maze import get_xyedges, find_zones, expand_line, speed_threshold
from plot_decode import get_combined, compare_lengths, get_proportions
from utils_plotting import plot_decoded_compare, plot_intersects, plot_zone

In [ ]:
import info.test as info

In [ ]:
data_filepath = 'C:/Users/Emily/code/emi_shortcut/cache/data'
pickle_filepath = 'C:/Users/Emily/code/emi_shortcut/cache/pickled'

In [ ]:
zone = dict()

u_start = Point(info.u_trajectory[0])
u_stop = Point(info.u_trajectory[-1])
u_line = LineString(info.u_trajectory)
zone['u'] = expand_line(u_start, u_stop, u_line, expand_by=1)

shortcut_start = Point(info.shortcut_trajectory[0])
shortcut_stop = Point(info.shortcut_trajectory[-1])
shortcut_line = LineString(info.shortcut_trajectory)
zone['shortcut'] = expand_line(shortcut_start, shortcut_stop, shortcut_line, expand_by=1)

novel_start = Point(info.novel_trajectory[0])
novel_stop = Point(info.novel_trajectory[-1])
novel_line = LineString(info.novel_trajectory)
zone['novel'] = expand_line(novel_start, novel_stop, novel_line, expand_by=1)

In [ ]:
for z in zone.keys():
    plot_zone(zone[z])

In [ ]:
position_time = np.linspace(0, 10, 11)

In [ ]:
position_y = np.ones(11) * 10
position_y += np.random.uniform(-0.1, 0.1, size=11)

In [ ]:
position_x = np.ones(11)
position_x[0:3] = position_x[0:3] * 15
position_x[3:6] = position_x[3:6] * 45
position_x[6:9] = position_x[6:9] * 75

In [ ]:
position_data = np.array([position_x, position_y]).T

In [ ]:
position = nept.Position(position_data, position_time)

In [ ]:
plt.plot(position.x, position.y, 'g.', ms=10)
for z in zone.keys():
    plot_zone(zone[z])
plt.ylim(8, 12)
plt.show()

In [ ]:
# spikes = [nept.SpikeTrain(np.ones(10), '01'),
#           nept.SpikeTrain(np.ones(5)*2, '02'),
#           nept.SpikeTrain(np.ones(10)*4, '01'),
#           nept.SpikeTrain(np.ones(5)*5, '02'),
#           nept.SpikeTrain(np.ones(10)*7, '01'),
#           nept.SpikeTrain(np.ones(5)*8, '02')]

broad = np.ones(10)
broad[:8] = broad[:8] * 5
broad[8:] = broad[8:] * 8

spikes = [nept.SpikeTrain(np.ones(5)*2, '02'),
          nept.SpikeTrain(np.ones(10)*4, '01'),
          nept.SpikeTrain(broad, '02')]
# spikes = [nept.SpikeTrain(np.array([5.]), '01')]

In [ ]:
lfp_time = position_time
lfp_data = np.ones(len(lfp_time))
lfp = nept.AnalogSignal(lfp_data, lfp_time)

In [ ]:
events = nept.Epoch(np.array([0., 10.]))

In [ ]:
filename = 'test-event.pkl'
pickled_path = os.path.join(data_filepath, filename)
with open(pickled_path, 'wb') as fileobj:
    pickle.dump(events, fileobj)
    
filename = 'test-lfp_swr.pkl'
pickled_path = os.path.join(data_filepath, filename)
with open(pickled_path, 'wb') as fileobj:
    pickle.dump(lfp, fileobj)
    
filename = 'test-lfp_theta.pkl'
pickled_path = os.path.join(data_filepath, filename)
with open(pickled_path, 'wb') as fileobj:
    pickle.dump(lfp, fileobj)
    
filename = 'test-position.pkl'
pickled_path = os.path.join(data_filepath, filename)
with open(pickled_path, 'wb') as fileobj:
    pickle.dump(position, fileobj)
    
filename = 'test-spike.pkl'
pickled_path = os.path.join(data_filepath, filename)
with open(pickled_path, 'wb') as fileobj:
    pickle.dump(spikes, fileobj)

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = get_xyedges(position)

In [ ]:
from analyze_tuning_curves import analyze
tuning_curve = analyze(info, speed_limit=0.0)

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)

for ii in range(len(tuning_curve)):
    print(ii)
    pp = plt.pcolormesh(xx, yy, tuning_curve[ii], cmap='pink_r')
    plt.colorbar(pp)
#     plt.axis('off')
    plt.show()

In [ ]:
from analyze_decode import get_decoded, analyze, point_in_zones
experiment_time = 'phase3'

In [ ]:
decode = get_decoded(info, tuning_curve, experiment_time, speed_limit=0.)

In [ ]:
yy = analyze(info, tuning_curve, experiment_time, speed_limit=0.)

In [ ]:
infos = [info]

experiment_times = ['phase3']
sessions = get_combined(infos, experiment_times)
norm_length = compare_lengths(infos, sessions)
proportion = get_proportions(norm_length)
plot_decoded_compare(proportion, ylabel='Proportion of time relative to track length')

In [ ]:
sessions[0]['phase3']['u'].time, sessions[0]['phase3']['shortcut'].time, sessions[0]['phase3']['novel'].time